# LIBRARY BLOCK

In [1]:
from selenium import webdriver
import json
import time
from datetime import datetime
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import pandas as pd

In [2]:
def get_table_data():
    result = {}
    head_ele_1st = driver.find_elements(By.CSS_SELECTOR, "thead tr:nth-child(1) td")
    head_1st_row = [ele.text for ele in head_ele_1st]

    head_ele_2nd = driver.find_elements(By.CSS_SELECTOR, "thead tr:nth-child(2) td")
    head_2nd_row = [ele.text for ele in head_ele_2nd]

    head_row = [
        head_1st_row[0],
        f"{head_1st_row[1]}_{head_2nd_row[0]}",
        f"{head_1st_row[1]}_{head_2nd_row[1]}",
        head_1st_row[2],
        f"{head_1st_row[3]}_{head_2nd_row[2]}",
        f"{head_1st_row[3]}_{head_2nd_row[3]}",
        f"{head_1st_row[4]}_{head_2nd_row[4]}",
        f"{head_1st_row[4]}_{head_2nd_row[5]}",
        f"{head_1st_row[5]}_{head_2nd_row[6]}",
        f"{head_1st_row[5]}_{head_2nd_row[7]}",
        f"{head_1st_row[5]}_{head_2nd_row[8]}"
    ]
    

    body_rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    for row in body_rows:
        cells = [cell.text.strip() for cell in row.find_elements(By.CSS_SELECTOR, "td")]
        if len(cells) == len(head_row):
            for i in range(len(head_row)):
                if head_row[i] in result:
                    result[head_row[i]].append(cells[i])
                else:
                    result[head_row[i]] = [cells[i]]
           
    return result


# CODE EXECUTION BLOCK

In [3]:
global driver
driver = webdriver.Chrome()
driver.get('https://cafef.vn/du-lieu/lich-su-giao-dich-hpg-1.chn')

In [4]:
#KHoong duoc xoa dong ben duoi
# df = pd.read_json('danh_sach_ma_simple.json', orient='values')
# codes_list = df.values.flatten().tolist()
# codes_list = [code.upper() for code in codes_list]



top_5_market_cap = ['HPG', 'STB', 'TCB', 'MBB', 'SSI']
codes_list = top_5_market_cap

start = 1
end = 5
df_market_stock_data = pd.DataFrame()
for idx,code in enumerate(codes_list[start:end]):
    print(f"Lấy dữ liệu cho mã cổ phiếu: {code}, {idx+1}/{end-start}")
    driver.get(f"https://cafef.vn/du-lieu/lich-su-giao-dich-{code}-1.chn")
    sleep(2)

    while True:         
        # --- BẮT ĐẦU ĐOẠN CODE THAY THẾ ---
        
        stock_data = [] # Khởi tạo rỗng để tránh lỗi biến chưa định nghĩa
        data_found = False # Cờ kiểm soát xem đã lấy được chưa
        start_wait_data = time.time() # Bấm giờ bắt đầu chờ dữ liệu
        
        print("    Đang đợi dữ liệu bảng xuất hiện...")
        
        # Vòng lặp chờ dữ liệu tối đa 3 phút (180s)
        while (time.time() - start_wait_data) < 180:
            try:
                temp_data = get_table_data()
                # Kiểm tra: Phải trả về list và list đó phải có phần tử
                if temp_data and len(temp_data) > 0:
                    stock_data = temp_data
                    data_found = True
                    print(f"    -> Đã lấy được {len(stock_data)} dòng dữ liệu.")
                    break # Thành công! Thoát vòng lặp chờ
            except Exception:
                # Nếu hàm get_table_data bị lỗi (do chưa load xong DOM), cứ bỏ qua và thử lại
                pass
            
            time.sleep(1) # Nghỉ 1 giây rồi thử lại để tránh treo máy
            
        page_next = driver.find_element(By.CSS_SELECTOR, 'div[class="pagination-item"] i[id="paging-right"]')
        if 'enable' in page_next.get_attribute('class'):
            print("không Có trang tiếp theo")
            break
        page_next.click()   
        sleep(2)

        
        # Nếu sau 3 phút mà vẫn không có dữ liệu
        if not data_found:
            print(f"-> LỖI: Không thể lấy dữ liệu bảng sau 3 phút. Dừng mã {code}.")
            continue # Thoát khỏi vòng lặp phân trang (while True lớn bên ngoài)

        # --- KẾT THÚC ĐOẠN CODE THAY THẾ ---

        df_stock_data = pd.DataFrame(stock_data)
        df_stock_data['Ma'] = code
        df_market_stock_data = pd.concat([df_market_stock_data, df_stock_data], ignore_index=True, axis=0)
        sleep(1)



current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
df_market_stock_data.to_csv(f'data/market_data/market_data_{current_time}.csv', index=False)
display(df_market_stock_data)


print("Dữ liệu của tất cả các công ty đã được lưu vào sheet 'TongHop' trong file 'du_lieu_2018_2023.xlsx'")


Lấy dữ liệu cho mã cổ phiếu: STB, 1/4
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 dòng dữ liệu.
    Đang đợi dữ liệu bảng xuất hiện...
    -> Đã lấy được 11 d

,Ngày,Giá (nghìn VNĐ)_Đóng cửa,Giá (nghìn VNĐ)_Điều chỉnh,Thay đổi,GD khớp lệnh_Khối lượng,GD khớp lệnh_Giá trị (tỷ VNĐ),GD thỏa thuận_Khối lượng,GD thỏa thuận_Giá trị (tỷ VNĐ),Giá (nghìn VNĐ)_Mở cửa,Giá (nghìn VNĐ)_Cao nhất,Giá (nghìn VNĐ)_Thấp nhất,Ma
0,12/12/2025,46,--,-1.7(-3.56 %),"8,811,500",421.35,"2,150,000",99,47.7,48.5,46,STB
1,11/12/2025,47.7,--,-0.9(-1.85 %),"6,162,200",296.71,0,0,48.4,48.6,47.7,STB
2,10/12/2025,48.6,--,0.05(0.10 %),"5,984,100",290.51,"110,000",5.67,48.45,48.65,48.4,STB
3,09/12/2025,48.55,--,-0.15(-0.31 %),"10,646,500",515.43,"6,627,000",315.24,48.35,49.25,48.05,STB
4,08/12/2025,48.7,--,-0.5(-1.02 %),"4,224,200",206.6,"1,312,000",64.76,49.2,49.25,48.7,STB
...,...,...,...,...,...,...,...,...,...,...,...,...
14955,28/12/2006,157,5.97,-6.3(-3.86 %),"162,400",25.79,0,0,166,167,150,SSI
14956,27/12/2006,160,6.09,6.1(3.96 %),"376,200",61.42,0,0,160,169.2,160,SSI
14957,26/12/2006,159,6.05,6.9(4.54 %),"435,500",67.04,0,0,137,162,137,SSI
14958,25/12/2006,152.1,5.79,-16.9(-10.00 %),"289,000",43.96,0,0,152.1,152.1,152.1,SSI


Dữ liệu của tất cả các công ty đã được lưu vào sheet 'TongHop' trong file 'du_lieu_2018_2023.xlsx'


In [26]:
import pandas as pd
df = pd.read_csv('data/market_data/market_data_20251214_001253.csv')
df.to_csv('data/market_data/market_data_20251214_001253_2.csv', index=False, encoding='utf-8-sig')
df = pd.read_csv('data/market_data/market_data_20251214_001253_2.csv')
print(df.iloc[2001]['Ngày'])
print(df.iloc[200]['Ngày'])


04/12/2017
27/02/2025


In [ ]:
from vnstock import Listing
import pandas as pd

# 1. Lấy dữ liệu (như code của bạn)
listing = Listing()
df = listing.all_symbols()

# --- LỰA CHỌN 1: Lưu thành danh sách đơn giản (Khuyên dùng) ---
# Kết quả trong file json sẽ là: ["AAA", "AAM", "AAT", ...]
# Dạng này nhẹ và dễ dùng nhất để loop qua các mã.
df["symbol"].to_json("df", orient="values")

# --- LỰA CHỌN 2: Lưu dạng cấu trúc Object ---
# Kết quả: [{"symbol": "AAA"}, {"symbol": "AAM"}, ...]
# Dạng này tốt nếu sau này bạn muốn lưu thêm thông tin khác (ví dụ: sàn, tên cty).
# force_ascii=False để không bị lỗi font tiếng Việt nếu có tên công ty
df[["symbol"]].to_json("danh_sach_ma_records.json", orient="records", indent=4, force_ascii=False)

print("Đã lưu file thành công!")

Đã lưu file thành công!


In [ ]:
import pandas as pd
df = pd.read_json('danh_sach_ma_simple.json', orient='values')
codes_list = df.values.flatten().tolist()
codes_list = [code.upper() for code in codes_list]


# 3. (Tùy chọn) Chuẩn hóa về chữ in hoa hết (đề phòng file nguồn có chữ thường)
print(codes_list)

['YTC', 'YEG', 'YBM', 'YBC', 'XPH', 'XMP', 'XMD', 'XMC', 'XLV', 'XHC', 'XDH', 'XDC', 'X77', 'X26', 'X20', 'WTC', 'WSS', 'WSB', 'WCS', 'VXT', 'VXP', 'VXB', 'VWS', 'VW3', 'VVS', 'VVN', 'VUA', 'VTZ', 'VTX', 'VTV', 'VTS', 'VTR', 'VTQ', 'VTP', 'VTO', 'VTM', 'VTL', 'VTK', 'VTJ', 'VTI', 'VTH', 'VTG', 'VTE', 'VTD', 'VTC', 'VTB', 'VTA', 'VST', 'VSP', 'VSN', 'VSM', 'VSI', 'VSH', 'VSG', 'VSF', 'VSE', 'VSC', 'VSA', 'VRG', 'VRE', 'VRC', 'VQC', 'VPW', 'VPS', 'VPR', 'VPL', 'VPI', 'VPH', 'VPG', 'VPD', 'VPC', 'VPB', 'VPA', 'VOS', 'VOC', 'VNZ', 'VNY', 'VNX', 'VNT', 'VNS', 'VNR', 'VNP', 'VNM', 'VNL', 'VNI', 'VNH', 'VNG', 'VNF', 'VNE', 'VND', 'VNC', 'VNB', 'VNA', 'VMT', 'VMS', 'VMK', 'VMI', 'VMG', 'VMD', 'VMC', 'VMA', 'VLW', 'VLP', 'VLG', 'VLF', 'VLC', 'VLB', 'VLA', 'VKP', 'VKD', 'VKC', 'VJC', 'VIX', 'VIW', 'VIT', 'VIS', 'VIR', 'VIP', 'VIN', 'VIM', 'VIH', 'VIG', 'VIF', 'VIE', 'VID', 'VIC', 'VIB', 'VHM', 'VHL', 'VHI', 'VHH', 'VHG', 'VHF', 'VHE', 'VHD', 'VHC', 'VGV', 'VGT', 'VGS', 'VGR', 'VGP', 'VGL', 'VGI'